# Experimentos de Maxwell - Cattaneo - Vernotte

## Modelo de MCV
## $$\dfrac{\partial T}{\partial t} + \tau \dfrac{\partial^2 T}{\partial t^2} = \alpha \dfrac{\partial^2 T}{\partial x^2}; \quad \dfrac{\partial T}{\partial t}(t + \tau) \simeq \dfrac{\partial T}{\partial t} + \tau \dfrac{\partial^2 T}{\partial t^2}$$

## $$q + \tau \dfrac{\partial q}{\partial t} = - \dfrac{\partial T}{\partial x}; \quad q (t + \tau) \simeq q + \tau \dfrac{\partial q}{\partial t}$$

## Con las variables causales
### $A = \dfrac{\partial T}{\partial t}, \quad B = \dfrac{\partial^2 T}{\partial x^2}$
### $C = q, \quad D  = \dfrac{\partial T}{\partial x}, \quad E = T$

La ecuación de transporte esta dada en su forma adimensional por medio de las siguientes variables adimensionales y parámetros:
* $T$ es la temperatura
* $t$ es el tiempo
* $x$ es la dirección espacial x
* $q$ es el flujo de calor
* El parámetro $\alpha$ es el número de Fourier $\alpha = D t_c L^{-2}$
* Con $t_c$ es el tiempo característico, $D$ es la difusividad térmica del material y $L$ es la longitud del material
* El parámetro $\tau$ es el tiempo efectivo de retardo $\tau = \Delta t_q t_c^{-1}$
* Con $\Delta t_q$ es el tiempo de retardo del flujo de calor

Este problema en particular se va a resolver para dos casos a las condiciones de frontera:

Determinista
* $\quad T(x = 1, t) = 1 \quad$ $T(x = 0, t) = \mu_\epsilon$

Estocástico
* $\quad T(x = 1, t) = 1 \quad$ $T(x = 0, t) = \epsilon(t)$ 

Con $\epsilon(t)$ ruido blanco Gaussiano y $\mu_\epsilon$ su promedio

Y la condición inicial para ambos casos
* $T(x, t = 0) = 1$

### Importación de bibliotecas y módulos

In [1]:
from IPython import display
import ipywidgets
import numpy as np               
import matplotlib.pyplot as plt  
import seaborn as sns
from numba import jit
import modulo_cattaneo as mcv

### Opciones de graficación
Estética de las gráficas

In [2]:
sns.set_style("darkgrid")
sns.set_context("paper", font_scale = 1.5)

## Función de simulación
Dependiendo de la frontera, se tiene la simulacion determinista o estocastica

In [3]:
@jit(nopython = True)
def simulacion(frontera):
    global nx, nt, alfa, tau, dx, dt, xpi
    # Condicion inicial
    u = np.zeros(nx + 1)
    flujocalor = np.zeros(nx + 1)
    temperatura = np.ones(nx + 1)
    # Inicializacion de las series de tiempo
    serie_a = np.empty(nt)
    serie_b = np.empty(nt)
    serie_c = np.empty(nt)
    serie_d = np.empty(nt)
    serie_e = np.empty(nt)
    # Proceso de la simulacion
    for n in range(nt):
        # Frontera. Al instante n se le asigna el valor correspondiente a la frontera
        temperatura[0] = frontera[n]
        # Calculo de u (derivada de la temperatura respecto al tiempo)
        u = mcv.rungekutta_u(u, temperatura, alfa, tau, dx, dt)
        # Calculo de la temperatura
        temperatura = mcv.calc_T(temperatura, u, dt)
        # Calculo del flujo de calor
        flujocalor = mcv.rungekutta_q(flujocalor, temperatura, tau, dx, dt)
        # Calculo del gradiente y laplaciano de la Temperatura, y laplaciano del flujo de calor
        lap_temperatura = mcv.d2dx2(temperatura, dx, full = True)
        grad_temperatura = mcv.ddx(temperatura, dx, full = True)
        # Captura de las series de tiempo
        serie_a[n] = u[xpi]
        serie_b[n] = lap_temperatura[xpi]
        serie_c[n] = flujocalor[xpi]
        serie_d[n] = grad_temperatura[xpi]
        serie_e[n] = temperatura[xpi]
    return serie_a, serie_b, serie_c, serie_d, serie_e

## Función de fluctuación
Construcción de la fluctuación de una variable del modelo

In [4]:
@jit(nopython = True)
def fluctuacion(determinista, estocastico):
    return estocastico - determinista

## Archivo del ensamble de realizaciones de la frontera

In [5]:
ensamble_fronteras = np.loadtxt("datos_frontera/fronteras.csv")
ensamble_fronteras.shape

(100, 100001)

## Dominio espacial

La solución numérica del problema se presenta solo en una dimensión espacial

In [6]:
nx = 256                        # Numero de pasos en el espacio
Lx = 1.0                        # Longitud del dominio espacial
x = np.linspace(0, Lx, nx + 1)  # Vector del dominio espacial, contiene nx+1 puntos incluyendo el 0
dx = x[1] - x[0]                # Tamaño de paso en el espacio, necesario para aplicar las diferencias finitas
dx

0.00390625

## Dominio temporal

El tiempo que se pretende dar solución al sistema de ecuaciones

In [7]:
nt = ensamble_fronteras.shape[1]  # Numero de pasos en el tiempo
Lt = 1.0                          # Longitud del dominio temporal
t = np.linspace(0, Lt, nt)        # Vector del dominio temporal, contiene nt+1 puntos incluyendo el 0
dt = t[1] - t[0]                  # Tamaño de paso en el tiempo, necesario para aplicar el método de Runge-Kutta mediante el solver
dt

1e-05

## Parámetros de la simulación

Para los arreglos

In [8]:
xpi = int(nx / 5)                 # Punto de interes donde se capturan las series de tiempo
nr = ensamble_fronteras.shape[0]  # Numero de realizaciones del ensamble
x[xpi]

0.19921875

Valores termodinámicos

In [9]:
difusividad = 1.0                         # Valor de referencia de la difusividad
t_c = (Lx * Lx) / difusividad             # Tiempo caracteriztico
num_alfa = difusividad * t_c / (Lx * Lx)  # Valor del parametro alfa
t_c

1.0

Tiempo efectivo de retardo

In [10]:
dtq = 8.0 * dt
tau_num = dtq / t_c
tau = tau_num * np.ones(nx + 1)
tau_num / dt

8.0

Número de Fourier

In [11]:
alfa = num_alfa * np.ones(nx + 1)  # Vector/arreglo alfa
alfa[0], alfa[-1] = 0, 0           # Extremos del vector/arreglo alfa

## Inicialización

In [12]:
fluc_a = np.empty((nr, nt))
fluc_b = np.empty((nr, nt))
fluc_c = np.empty((nr, nt))
fluc_d = np.empty((nr, nt))
fluc_e = np.empty((nr, nt))
serie_a = np.empty((nr, nt))
serie_b = np.empty((nr, nt))
serie_c = np.empty((nr, nt))
serie_d = np.empty((nr, nt))
serie_e = np.empty((nr, nt))

## Proceso principal

In [13]:
# Barra de progreso
progreso = ipywidgets.FloatProgress(
    value = 0,
    min = 0,
    max = nr, 
    description = 'Progreso:')
display.display(progreso)

progreso.value = 0
for r in range(nr):
    # Fronteras
    frontera_est = ensamble_fronteras[r]
    frontera_det = np.mean(frontera_est) * np.ones(nt)
    # Simulaciones
    a_est, b_est, c_est, d_est, e_est = simulacion(frontera_est)
    a_det, b_det, c_det, d_det, e_det = simulacion(frontera_det)
    # Fluctuaciones
    fluc_a[r] = fluctuacion(a_det, a_est)
    fluc_b[r] = fluctuacion(b_det, b_est)
    fluc_c[r] = fluctuacion(c_det, c_est)
    fluc_d[r] = fluctuacion(d_det, d_est)
    fluc_e[r] = fluctuacion(e_det, e_est)
    # Series
    serie_a[r] = a_est
    serie_b[r] = b_est
    serie_c[r] = c_est
    serie_d[r] = d_est
    serie_e[r] = e_est
    # Progreso del trabajo
    progreso.value = r + 1

FloatProgress(value=0.0, description='Progreso:')

## Archivos

In [14]:
ruta = "datos_experimentos_cattaneo/"
np.savetxt(ruta + "a_serie.csv", serie_a)
np.savetxt(ruta + "b_serie.csv", serie_b)
np.savetxt(ruta + "c_serie.csv", serie_c)
np.savetxt(ruta + "d_serie.csv", serie_d)
np.savetxt(ruta + "e_serie.csv", serie_e)
np.savetxt(ruta + "a_fluc.csv", fluc_a)
np.savetxt(ruta + "b_fluc.csv", fluc_b)
np.savetxt(ruta + "c_fluc.csv", fluc_c)
np.savetxt(ruta + "d_fluc.csv", fluc_d)
np.savetxt(ruta + "e_fluc.csv", fluc_e)